In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import rasterio
from rasterio.plot import show
import xarray as xr
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [6]:

# 1. Load GeoJSON data (road_transport_interurban_CAT)
roads = gpd.read_file('../road_transport_interurban_CAT.geojson')

In [ ]:
# 2. Load shapefiles (population grid and administrative divisions)
population = gpd.read_file('../Poblacio/gridpoblacio01012022.shp')

In [9]:
# 3. Load road links shapefiles
road_links = gpd.read_file('../OpenTransportMap/Barcelona/roadlinks_ES511.shp')

In [10]:
administrative_divisions = gpd.read_file('../LimitsAdministratius/Comarques/divisions-administratives-v2r1-comarques-5000-20240705.shp')

In [39]:
!python -m pip install "xarray[complete]" 




   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 882.6 kB/s eta 0:00:12
   -- ------------------------------------- 0.8/11.0 MB 985.5 kB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.0 MB 931.8 kB/s eta 0:00:11
   ---- ----------------------------------- 1.3/11.0 MB 986.4 kB/s eta 0:00:10
   ----- ---------------------------------- 1.6/11.0 MB 1.0 MB/s eta 0:00:10
   ------ --------------------------------- 1.8/11.0 MB 1.0 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.0 MB 1.1 MB/s eta 0:00:09
   -------- ------------------------------- 2.4/11.0 MB 1.1 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.0 MB 1.1 MB/s eta 0:00:08
   --------- ------

In [40]:
import xarray as xr
print(xr.backends.list_engines())


{'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [38]:
# 4. Load NO2 data (NetCDF format)
no2_data = xr.open_dataset('../hourly/sconcno2/sconcno2_2021012700.nc', engine='netcdf4')

ValueError: unrecognized engine 'netcdf4' must be one of your download engines: ['scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [ ]:

# Define a base map for visualization
catalonia_map = folium.Map(location=[41.8245, 1.5920], zoom_start=8)


DriverError: ../OpenTransportMap/Girona/divisions-administratives-v2r1-comarques-5000-20240705.cpg: No such file or directory

In [ ]:

# Add roads to the map
for _, row in roads.iterrows():
    coords = [(lat, lon) for lon, lat, _ in row.geometry[0]]
    folium.PolyLine(coords, color='blue', weight=2.5, opacity=0.7).add_to(catalonia_map)

# Add population data
folium.Choropleth(
    geo_data=population,
    data=population,
    columns=['GRID_ID', 'POPULATION'],
    key_on='feature.properties.GRID_ID',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population Density'
).add_to(catalonia_map)

# Add administrative boundaries
for _, row in administrative_divisions.iterrows():
    folium.Polygon(
        locations=[(lat, lon) for lon, lat in row.geometry.exterior.coords],
        color='green',
        weight=1,
        fill=True,
        fill_opacity=0.3
    ).add_to(catalonia_map)

# Extract NO2 grid and add heatmap
no2_array = no2_data['no2'].isel(time=0).values
lat = no2_data['lat'].values
lon = no2_data['lon'].values
no2_points = [(lat[i], lon[j], no2_array[i, j]) for i in range(len(lat)) for j in range(len(lon))]

HeatMap(
    data=[(p[0], p[1], p[2]) for p in no2_points],
    max_zoom=16,
    radius=15
).add_to(catalonia_map)



# Show Folium map
catalonia_map.save('catalonia_map.html')

# Save static map using Matplotlib
fig, ax = plt.subplots(figsize=(10, 10))
administrative_divisions.plot(ax=ax, color='none', edgecolor='black', linewidth=0.5)
roads.plot(ax=ax, color='blue', linewidth=0.5)
population.plot(ax=ax, column='POPULATION', cmap='YlOrRd', legend=True)
plt.title('Catalonia NO2 and Population Map')
plt.show()
